# Integrantes

* Juan Ignacio García (a2008)
* Rodrigo Mesa Marchi (a2016)
* Danilo Simón Reitano Andrades (a2020)

In [40]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi
from queue import PriorityQueue
from aima_libs.tree_hanoi import NodeHanoi


In [62]:
def a_star_evaluation(node: NodeHanoi) -> int:
    """
    Path cost + heuristic which prioritizes disks in the right position weighted by size
    """
    return node.path_cost - sum(node.state.get_state()[2])

def greedy_evaluation(node: NodeHanoi) -> int:
    """
    Heuristic which prioritizes disks in the right position weighted by size
    """
    return -sum(node.state.get_state()[2])

def dijkstra_evaluation(node: NodeHanoi) -> int:
    """
    Path cost
    """
    return node.path_cost

def search_algo(number_disks, evaluation_function):
    # Inicializamos el problema
    list_disks = [i for i in range(number_disks, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    initial_node = NodeHanoi(problem.initial)

    # Creamos una cola FIFO con el nodo inicial
    frontier = PriorityQueue()
    frontier.put((evaluation_function(initial_node), initial_node))

    # Creamos el set con estados ya visitados
    explored = set()
    
    node_explored = 0
    
    while not frontier.empty():
        node = frontier.get()[1]
        node_explored += 1
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": frontier.qsize(),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node.expand(problem):
            # Solo si el estado del nodo no fue explorado
            if next_node.state not in explored:
                frontier.put((evaluation_function(next_node), next_node))

    # Si no se encontro la solución, devolvemos la métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": frontier.qsize(),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return problem, metrics

In [63]:
%%timeit
problem_a_star, metrics_a_star = search_algo(7, a_star_evaluation)


108 ms ± 803 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
%%timeit
problem_greedy, metrics_greedy = search_algo(7, greedy_evaluation)

108 ms ± 538 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
%%timeit
problem_dij, metrics_dij = search_algo(7, dijkstra_evaluation)

172 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [79]:
_, metrics_a_star = search_algo(7, a_star_evaluation)
print(metrics_a_star)
print(f"Diferencia con la solución óptima: {metrics_a_star['cost_total']-(2**7-1)}")

{'solution_found': True, 'nodes_explored': 2509, 'states_visited': 1494, 'nodes_in_frontier': 48, 'max_depth': 127, 'cost_total': 127.0}
Diferencia con la solución óptima: 0.0


In [80]:
_, metrics_greedy = search_algo(7, greedy_evaluation)
print(metrics_greedy)
print(f"Diferencia con la solución óptima: {metrics_greedy['cost_total']-(2**7-1)}")

{'solution_found': True, 'nodes_explored': 2547, 'states_visited': 1449, 'nodes_in_frontier': 95, 'max_depth': 138, 'cost_total': 138.0}
Diferencia con la solución óptima: 11.0


In [81]:
_, metrics_dij = search_algo(7, dijkstra_evaluation)
print(metrics_dij)
print(f"Diferencia con la solución óptima: {metrics_dij['cost_total']-(2**7-1)}")

{'solution_found': True, 'nodes_explored': 4071, 'states_visited': 2072, 'nodes_in_frontier': 208, 'max_depth': 127, 'cost_total': 127.0}
Diferencia con la solución óptima: 0.0
